# Adding measurements to Frames

This notebook will outline the general process to add measurement data to frames to allow it to be sent over the network to clients.
Later, it will also cover the means by which we can add automatic measurement calculations and attach these to our sent frames.

## First, create the server/client pair

In [1]:
from nanover.app import NanoverImdApplication
from nanover.websocket import NanoverImdClient
from nanover.websocket.client.app_client import get_websocket_address_from_app_server

# Create server/client pair.
server = NanoverImdApplication.basic_server(port=0)
client = NanoverImdClient.from_url(get_websocket_address_from_app_server(server))

First we will need to load a simulation we want to visualise and compute metrics for.

In [2]:
import MDAnalysis as mda

# Use MDAnalysis to create a nanover simulation to run.
simulation = mda.Universe("../mdanalysis/serotonine_receptor.pdb")

For this tutorial we will compute the distance between a couple of atoms and the radius of gyration of the protein. Then store this information in their relevant NanoVer implementations.
NanoVer provides implementations for 4 measurement types:
- `Scalar`: any scalar metric not relating to geometric information.
- `Distance`: for distances between pairs of atoms.
- `Angle`: for angles between any triplet of atoms.
- `Dihedral`: for torsions between and quadruple of atoms.

In [3]:
import numpy as np
from openmm.unit import angstrom
from nanover.trajectory.measure import *  # OK to wildcard import here.

rad_gyr = Scalar(
    "backbone_rg", simulation.select_atoms("protein and backbone").radius_of_gyration(),
    unit=angstrom
)

atom1, atom2 = simulation.select_atoms("(resnum 75 or resnum 100) and name CA")
distance = Distance(
    "d1",
    atom1.index,
    atom2.index,
    np.linalg.norm(atom1.position - atom2.position),  # Compute distance
    unit=angstrom
)

Next we will create a `MeasureCollection` object to store all of the desired measurements and then add these measurements to a frame containing also containing the structure of the protein.

In [4]:
from nanover.trajectory.measure_collections import MeasureCollection

collection = MeasureCollection(scalars=[rad_gyr], distances=[distance])

print(collection)

MeasureCollection containing: 1 scalar, 1 distance, 0 angle, and 0 dihedral measurements.


In [5]:
from nanover.mdanalysis import mdanalysis_to_frame_data

frame = mdanalysis_to_frame_data(simulation, topology=True, positions=True)
collection.add_to_framedata(frame)

server.frame_publisher.send_frame(frame)

Now we can verify that the sent data does, in fact, contain our desired metrics and can visualise the PDB.

In [6]:
client_measures = MeasureCollection.from_framedata(client.current_frame)
client_measures

MeasureCollection containing:
	1 Scalars; <(Scalar) backbone_rg: 28.960801778835663 >
	1 Distances; <(Distance) d1: 10.694733619689941 >

## Cleanup

In [7]:
# Close the open server/client ports.
server.close()
client.close()